In [ ]:
# CONVERT TO TFLITE
# Check if model is TFLite compatible
if best_classifier.__class__.__name__ != 'LogisticRegression':
    print(f"\nWARNING: {best_classifier.__class__.__name__} is not directly TFLite compatible!")
    print("Only Logistic Regression can be fully converted to TFLite.")
    
    # Create a TFLite-compatible version with Logistic Regression
    lr_model_path = os.path.join(MODELS_DIR, 'logistic_regression_model.pkl')
    lr_classifier = joblib.load(lr_model_path)
    print(f"Loaded Logistic Regression model")
    
    # Build LR version
    lr_full_model = YAMNetClassifierModel(yamnet_model, scaler, lr_classifier, label_encoder)
    print("Created Logistic Regression pipeline")
    
    # Save LR version
    lr_saved_model_path = os.path.join(MODELS_DIR, 'full_pipeline_lr_savedmodel')
    tf.saved_model.save(lr_full_model, lr_saved_model_path)
    print(f"Saved LR model to: {lr_saved_model_path}")
    
    model_to_convert = lr_full_model
    conversion_note = "logistic_regression"
else:
    model_to_convert = full_model
    conversion_note = best_model_name.lower()

print(f"\nConverting {conversion_note} model to TFLite...")

# Create converter
converter = tf.lite.TFLiteConverter.from_concrete_functions([
    model_to_convert.__call__.get_concrete_function(
        tf.TensorSpec(shape=[15360], dtype=tf.float32)
    )
])

In [ ]:
# Optimization options

# 1. Default (no optimization)
print("\n1. Default (Float32, no optimization)")
tflite_model = converter.convert()
tflite_path = os.path.join(TFLITE_DIR, f'model_{conversion_note}_float32.tflite')
with open(tflite_path, 'wb') as f:
    f.write(tflite_model)
print(f"   Saved: {tflite_path}")
print(f"   Size: {len(tflite_model) / 1024:.2f} KB")

# 2. Dynamic range quantization
print("\n2. Dynamic range quantization (Int8 weights, Float32 activations)")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized = converter.convert()
tflite_quant_path = os.path.join(TFLITE_DIR, f'model_{conversion_note}_dynamic_quant.tflite')
with open(tflite_quant_path, 'wb') as f:
    f.write(tflite_quantized)
print(f"   Saved: {tflite_quant_path}")
print(f"   Size: {len(tflite_quantized) / 1024:.2f} KB")
print(f"   Compression: {(1 - len(tflite_quantized)/len(tflite_model)) * 100:.1f}%")


In [ ]:
# TEST TFLITE MODELS
def test_tflite_model(model_path, test_audio_np):
    """Test a TFLite model"""
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Set input
    interpreter.set_tensor(input_details[0]['index'], test_audio_np)
    
    # Run inference
    interpreter.invoke()
    
    # Get outputs
    results = {}
    for output in output_details:
        results[output['name']] = interpreter.get_tensor(output['index'])
    
    return results

# Generate test audio
test_audio_np = np.random.randn(15360).astype(np.float32)

print("\nTesting Float32 model...")
try:
    results_float = test_tflite_model(tflite_path, test_audio_np)
    print("Float32 model works!")
    print(f"  Probabilities shape: {results_float['probabilities'].shape}")
    print(f"  Predicted class: {results_float['predicted_class_id']}")
except Exception as e:
    print(f"Error: {e}")

print("\nTesting Quantized model...")
try:
    results_quant = test_tflite_model(tflite_quant_path, test_audio_np)
    print("Quantized model works!")
    print(f"  Probabilities shape: {results_quant['probabilities'].shape}")
    print(f"  Predicted class: {results_quant['predicted_class_id']}")
    
    # Compare outputs
    if 'probabilities' in results_float and 'probabilities' in results_quant:
        prob_diff = np.abs(results_float['probabilities'] - results_quant['probabilities']).max()
        print(f"\nQuantization impact:")
        print(f"  Max probability difference: {prob_diff:.6f}")
        if prob_diff < 0.01:
            print("  Minimal impact from quantization")
        else:
            print("  Moderate impact from quantization")
except Exception as e:
    print(f"✗ Error: {e}")

# REAL AUDIO TEST 
try:
    # Load a real test sample
    test_features = np.load(os.path.join('../data/approach1/features', 'yamnet_features.npy'))
    test_labels = np.load(os.path.join('../data/approach1/features', 'yamnet_labels.npy'))
    test_metadata = pd.read_csv(os.path.join('../data/approach1/features', 'yamnet_features_metadata.csv'))
    
    # Pick a random sample
    idx = np.random.randint(len(test_features))
    sample_audio_path = test_metadata.iloc[idx]['frame_path']
    true_label = test_labels[idx]
    
    print(f"\nTesting on: {os.path.basename(sample_audio_path)}")
    print(f"True label: {true_label}")
    
    # Load audio
    sample_audio = np.load(sample_audio_path).astype(np.float32)
    
    # Test with TFLite
    results = test_tflite_model(tflite_quant_path, sample_audio)
    pred_class_id = results['predicted_class_id'][0]
    pred_class_name = classes[pred_class_id]
    confidence = results['probabilities'][0][pred_class_id]
    
    print(f"\nPrediction:")
    print(f"  Class: {pred_class_name}")
    print(f"  Confidence: {confidence:.4f}")
    print(f"  Correct: {'✓' if pred_class_name == true_label else '✗'}")
    
    print(f"\nAll probabilities:")
    for i, cls in enumerate(classes):
        prob = results['probabilities'][0][i]
        marker = " ←" if i == pred_class_id else ""
        print(f"  {cls:15s}: {prob:.4f}{marker}")
        
except Exception as e:
    print(f"Could not test on real audio: {e}")

# DEPLOYMENT GUIDE
print("\n" + "="*70)
print("DEPLOYMENT GUIDE")
print("="*70)

deployment_info = f"""
Models saved in: {TFLITE_DIR}

Available models:
1. model_{conversion_note}_float32.tflite - Full precision (larger, more accurate)
2. model_{conversion_note}_dynamic_quant.tflite - Quantized (smaller, faster, slight accuracy loss)

INPUT REQUIREMENTS:
- Audio format: 16kHz, mono, float32
- Duration: EXACTLY 0.96 seconds (15,360 samples)
- Amplitude range: [-1.0, 1.0] (normalized)

For variable-length audio:
1. Resample to 16kHz
2. Split into 0.96s segments (with overlap if needed)
3. Run inference on each segment
4. Aggregate results (e.g., max probability, voting)

OUTPUT:
- probabilities: [5] array of class probabilities
- predicted_class_id: int (0-4)
- predicted_class_name: string
- confidence: float (max probability)

Classes: {', '.join(classes)}

PERFORMANCE:
- Best model: {best_model_name}
- Test F1-Score: {comparison_df.iloc[0]['Test F1']:.4f}
- Test Accuracy: {comparison_df.iloc[0]['Test Acc']:.4f}

INTEGRATION EXAMPLES:
- Python: Use tf.lite.Interpreter
- Android: Use TensorFlow Lite Android library
- iOS: Use TensorFlow Lite iOS library
- Web: Use TensorFlow.js (convert from SavedModel)
- Edge devices: TFLite runtime

See TensorFlow Lite documentation for platform-specific integration.
"""

print(deployment_info)

# Save deployment guide
guide_path = os.path.join(TFLITE_DIR, 'DEPLOYMENT_GUIDE.txt')
with open(guide_path, 'w') as f:
    f.write(deployment_info)
print(f"\n✓ Saved deployment guide to: {guide_path}")

print("\n" + "="*70)
print("PIPELINE BUILD COMPLETE")
print("="*70)
print("All models saved and tested")
print("Ready for deployment!")